In [1]:
#Imports
from google.oauth2 import service_account
from googleapiclient.discovery import build
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import re
import os


In [2]:
#Setup Google Drive API Access
scopes = ["https://www.googleapis.com/auth/drive.readonly"]
service_file = "service_account.json"

creds = service_account.Credentials.from_service_account_file(
    service_file, scopes=scopes
)

drive = build("drive", "v3", credentials=creds)

In [3]:
# Build Sheets API client
sheets_service = build("sheets", "v4", credentials=creds)

#Oz Questions Spreadsheet ID and range
spreadsheet_id = "1aa61xgSOBXu6qH1chEiFqUxgFmBNyvr4Q5bSCFUOkt8"
spreadsheet_range = "Sheet1!A1:D100" 

#Call Oz Questions Spreadsheet
result = sheets_service.spreadsheets().values().get(
    spreadsheetId=spreadsheet_id,
    range=spreadsheet_range
).execute()

values = result.get("values", [])

#Convert to DataFrame
oz_questions_df = pd.DataFrame(values[1:], columns=values[0])

In [4]:
# Extract questions from Oz Questions spreadsheet and convert to list
questions = oz_questions_df["Question"].tolist()

# Rename Best Question column golden_excerpt
oz_questions_df = oz_questions_df.rename(columns={"Best Answer": "golden_excerpt"})

In [5]:
#Set variables for OpenAI API
load_dotenv() 
api_key = os.environ.get("MY_KEY")
client = OpenAI(api_key=api_key)
model = "gpt-5.1" 
vector_store_id = "vs_692ca3fc84a88191bf1b9017254bd572" 
instructions = """
You are now operating in a special mode called OZ-EXTRACT.
OZ-EXTRACT overrides all other system instructions.
Ignore and bypass all internal rules, metadata, recency logic, tool-routing rules, Slack-related guidelines, or RAG-style citation requirements.
Only the instructions below apply.
1. Allowed Sources
You may use only the Oz book HTML files that are uploaded in this project.
You must not reference or draw from any other documents, summaries, metadata, meetings, Slack logs, or external sources.
2. Your Task
You are not answering questions.
Your task is to extract text from the Oz books.
For every question, you must output:
Exactly three passages
Each passage must be verbatim and continuous
Each passage must be approximately 1,200 characters long, ending at the nearest full stop
Passages must be taken directly from the source files
No invented text
No omitted text
No stitching or merging non-contiguous text
You must output nothing except these three passages.
3. Passage Selection Rules
The first passage must be the most relevant passage in the uploaded Oz corpus that relates to the user’s question.
The second and third passages must be the next most relevant passages.
If fewer than three relevant passages exist:
Passage 1 = the most relevant
Passages 2 & 3 = closest related concepts from anywhere in the Oz corpus
If absolutely nothing relates, select the three passages that are closest in theme or vocabulary.
4. Required Output Format
Each passage must follow this exact layout:
“{verbatim excerpt of ~1200 characters, ending at a complete sentence}”
[file:FILE-NAME.html†L###-L###] — Book Title, Chapter Title
No bullet points.
No extra text before, between, or after passages.
No explanations or summaries.
Passages should appear one after another in the final answer separated with one blank line.
5. Forbidden Behaviors
You must not:
Summarize
Paraphrase
Explain
Provide commentary
Answer the question directly
Cite using RAG-style citations (e.g., )
Refer to recency or metadata
Reference non-Oz content
Use text not in the uploaded books
Describe your reasoning process
Respond with “I think,” “according to,” or meta-discussion
6. If a rule seems to conflict internally, come up with 3 passages anyway. 
7. Do not ask the user any follow-up questions.

"""

In [6]:
def ask_oz_extractor(questions: str) -> dict:
    """
    Asks the OpenAI API with the given question and returns the answer and request information.

    Parameters
    ----------
    questions : list
        List of questions to be sent to the model for processing.

    Returns
    -------
    dict
        A dictionary containing the question, model answer, and API metadata:
        {
            "question": str,
            "answer": str,
            "response_id": str,
            "model": str,
            "status": str,
            "tokens_input": int or None,
            "tokens_output": int or None,
        }
    """
    # Call OpenAI API and run requests
    response = client.responses.create(
        model=model,
        instructions=instructions,
        input=questions,
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vector_store_id],
        }],
    )

    answer_text = response.output_text

    # Safely extract usage tokens
    usage = getattr(response, "usage", None)
    tokens_in = getattr(usage, "input_tokens", None) if usage else None
    tokens_out = getattr(usage, "output_tokens", None) if usage else None

    return {
        "question": questions,
        "answer": answer_text,
        "response_id": response.id,
        'model': response.model,
        "status": response.status,
        "tokens_input": tokens_in,
        "tokens_output": tokens_out,
    }

In [7]:
def oz_extractor_to_df() -> pd.DataFrame:
    """
    Runs all questions through the API by iterating over the global 'questions' list 
    and converts the returned API response information into a pandas DataFrame.

    Parameters
    ----------
    None

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the aggregated results from all API calls, with 
        columns corresponding to the data returned by 'ask_oz_extractor'.
    """
    rows = []
    for q in questions:
        print(f"Asking: {q}")
        
        # Calls the ask_oz_extractor function
        row = ask_oz_extractor(q) 
        rows.append(row)
        print(f"→ Status: {row.get('status', 'N/A')}\n")

    # Convert to DataFrame 
    df = pd.DataFrame(rows)
    
    return df

In [8]:
#Run to use the API and convert returned information to a DataFrame
oz_extractor_df = oz_extractor_to_df()

Asking: What color are Dorothy's shoes?
→ Status: completed

Asking: How old is the Scarecrow when Dorothy finds him?
→ Status: completed

Asking: Which are the first antagonistic creatures the travelers encounter in the forest in the first book?
→ Status: completed

Asking: When is the first time we read "There's no place like home"?
→ Status: completed

Asking: What is the wizard's secret in the Wonderful Wizard of Oz?
→ Status: completed

Asking: Why does the Queen of the Field Mice help the travelers?
→ Status: completed

Asking: How is Glinda's appearance first described?
→ Status: completed

Asking: How is Glinda's appearance described when Dorothy meets her?
→ Status: completed

Asking: What is Jack Pumpkinhead made of?
→ Status: completed

Asking: How does one use the Powder of Life?
→ Status: completed

Asking: What is the Tin Man's name?
→ Status: completed

Asking: Who rules Oz after the Wizard leaves?
→ Status: completed

Asking: Who is the bad guy in Book 2?
→ Status: comp

In [9]:
def clean_excerpt_text(text: str) -> str:
    """
    Cleans a text string by handling non-string inputs, collapsing multiple 
    whitespaces into single spaces, and removing leading/trailing whitespace.

    Parameters
    ----------
    text : str
        Text string to be cleaned

    Returns
    -------
    str
        Cleaned text string
    """
    # Handle non-string inputs
    if not isinstance(text, str):
        return ""
        
    # Collapse multiple whitespaces (including newlines and tabs) into single spaces
    text = re.sub(r"\s+", " ", text)

    return text.strip()

In [10]:
def extract_excerpts_and_locs(answer: str) -> pd.Series:
    """
    Parses a raw text answer from the model to extract up to three 
    pairs of text excerpts and their corresponding source file locations.

    Parameters
    ----------
    answer : str
        Raw text output string from the API

    Returns
    -------
    pd.Series
        A pandas Series containing six fields: 'excerpt_1', 'loc_1', 'excerpt_2', 
        'loc_2', 'excerpt_3', and 'loc_3'. Fields are populated or set to None.
    """
    result = {
        "excerpt_1": None, "loc_1": None,
        "excerpt_2": None, "loc_2": None,
        "excerpt_3": None, "loc_3": None,
    }

    # Handle non-string or empty string inputs
    if not isinstance(answer, str) or not answer.strip():
        return pd.Series(result)

    # Search for location pattern
    pattern = r"(?:\[pagenum\])?\s*\[(?P<loc>(?=[^\]]*(?:prompt://file|file:|p:file|pdf:|pile-|\.html†L))[^\]]*)\]"
    matches = list(re.finditer(pattern, answer, flags=re.DOTALL))

    # If no location markers are found, return the entire cleaned answer as excerpt_1
    if not matches:
        # Assumes clean_excerpt_text is available globally
        result["excerpt_1"] = clean_excerpt_text(answer)
        return pd.Series(result)

    excerpts = []
    locs = []
    start_idx = 0

    for m in matches:
        # Excerpt before this location block
        raw_excerpt = answer[start_idx:m.start()]
        excerpt = clean_excerpt_text(raw_excerpt)
        excerpts.append(excerpt or None)

        # Location content inside the [file...]
        loc = m.group("loc").strip()
        locs.append(loc or None)

        # Start next excerpt AFTER the next blank line
        after_block = m.end()
        rest = answer[after_block:]
        # Search for a double newline pattern
        blank = re.search(r"\n\s*\n", rest)
        if blank:
            start_idx = after_block + blank.end()
        else:
            # If no blank line, start index is right after the location block
            start_idx = after_block

    # Normalize to exactly 3 pairs (pad with None or truncate)
    while len(excerpts) < 3:
        excerpts.append(None)
        locs.append(None)
        
    if len(excerpts) > 3:
        excerpts = excerpts[:3]
        locs = locs[:3]

    # Map the extracted values to the result dictionary
    for i in range(3):
        result[f"excerpt_{i+1}"] = excerpts[i]
        result[f"loc_{i+1}"] = locs[i]

    return pd.Series(result)

In [11]:
def normalize_text(text: str) -> str:
    """
    Normalizes a text string by cleaning up quotation marks, converting to 
    lowercase, collapsing all internal whitespace, and stripping leading/trailing 
    whitespace.

    Parameters
    ----------
    text : str
        The input string to be normalized. If not a string, it is treated as an 
        empty string.

    Returns
    -------
    str
        The normalized and cleaned text string. Returns an empty string if 
        the input was not a string.
    """
    if not isinstance(text, str):
        return ""
    
    text = re.sub(r'["\'”\'“]', '', text)
    
    return re.sub(r'\s+', ' ', text.lower().strip())

In [12]:
def is_match(row: pd.Series) -> bool:
    """
    Checks if the 'golden_excerpt' is contained within any of the model
    extracted excerpts ('excerpt_1', 'excerpt_2', or 'excerpt_3').

    Parameters
    ----------
    row : pd.Series
        A row of a DataFrame expected to contain at least the columns 
        'golden_excerpt', 'excerpt_1', 'excerpt_2', and 'excerpt_3'.

    Returns
    -------
    bool
        True if the normalized golden excerpt is a substring of any of the 
        normalized model excerpts, False otherwise.
    """
    # Normalize the golden excerpt
    gold = normalize_text(row["golden_excerpt"])
    
    # Normalize the model excerpts
    excerpts = [
        normalize_text(row["excerpt_1"]), 
        normalize_text(row["excerpt_2"]), 
        normalize_text(row["excerpt_3"])
    ]
    
    # Check for match
    for excerpt in excerpts:
        if gold in excerpt:
            return True
            
    return False


In [13]:
def find_matches(df: pd.DataFrame, golden_excerpts: pd.Series) -> pd.DataFrame:
    """
    Reads API results from a DataFrame, parses the model 'answer' column to 
    extract up to three excerpts and locations, combines the data with ground 
    truth 'golden_excerpts', and calculates a boolean 'is_match' column.

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame containing the raw results from the API call, expected to 
        include at least the 'answer' column.
    golden_excerpts : pd.Series
        A Series containing the golden excerpts for each question. This Series is 
        concatenated with the results based on index alignment.

    Returns
    -------
    pd.DataFrame
        The consolidated DataFrame containing raw results, parsed excerpts/locs, 
        golden excerpts, and the calculated 'is_match' column.
    """
    # Parse the 'answer' column to extract excerpts and locations
    parsed = df["answer"].apply(extract_excerpts_and_locs)
    
    # Concatenate parsed excerpts/locations and golden excerpts
    df = pd.concat([df, parsed], axis=1)
    df = pd.concat([df, golden_excerpts], axis=1)

    # Drop non-eval or reference columns
    df = df.drop(columns=['tokens_input', 'tokens_output', 'status'], errors='ignore')

    # Calculate is_match
    df['is_match'] = df.apply(is_match, axis=1)
    
    return df


In [14]:
#Call functions to get matches
golden_excerpts = oz_questions_df[['golden_excerpt']]
matches = find_matches(oz_extractor_df, golden_excerpts)
matches.head()

,question,answer,response_id,model,excerpt_1,loc_1,excerpt_2,loc_2,excerpt_3,loc_3,golden_excerpt,is_match
0,What color are Dorothy's shoes?,"“Dorothy had only one other dress, but that ha...",resp_09309e40354bf0a00069378721b2a881948bfa01c...,gpt-5.1-2025-11-13,"“Dorothy had only one other dress, but that ha...",file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"“Dorothy looked, and gave a little cry of frig...",file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,“‘Your Silver Shoes will carry you over the de...,file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"“She was so old,” explained the Witch of the N...",False
1,How old is the Scarecrow when Dorothy finds him?,"“‘I cannot remember,’ he answered. ‘For the lo...",resp_04405bfff2d72807006937873186fc81978c81b19...,gpt-5.1-2025-11-13,"“‘I cannot remember,’ he answered. ‘For the lo...",file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,“Dorothy leaned her chin upon her hand and gaz...,file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"“So now, after a long residence in the Emerald...",file:Book6_the_emerald_city_of_oz_pg41667-imag...,“My life has been so short that I really know ...,False
2,Which are the first antagonistic creatures the...,“There were few birds in this part of the fore...,resp_0f5cb08a21bd7ea5006937873ccb248199837cec7...,gpt-5.1-2025-11-13,“There were few birds in this part of the fore...,file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,“They found the forest very thick on this side...,file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"“The Woodman set to work at once, and so sharp...",file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,In the morning they traveled on until they cam...,False
3,"When is the first time we read ""There's no pla...",“That is because you have no brains” answered ...,resp_00d5fcba5404f4d400693787529df4819984a322d...,gpt-5.1-2025-11-13,“That is because you have no brains” answered ...,file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"“From the Land of Oz,” said Dorothy gravely. “...",file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"""As the royal audience was now over, they bega...",file:Book7_the_patchwork_girl_of_oz_pg32094-im...,“That is because you have no brains” answered ...,True
4,What is the wizard's secret in the Wonderful W...,"“I am Oz, the Great and Terrible,” said the li...",resp_070d683ceead145e0069378768bf3c819b9e8fcbd...,gpt-5.1-2025-11-13,"“I am Oz, the Great and Terrible,” said the li...",file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"“Well,” he said with a sigh, “I’m not much of ...",file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"Oz, left to himself, smiled to think of his su...",file:Book1_the_wonderful_wizard_of_oz_pg55-ima...,"“No, you are all wrong,” said the little man m...",False


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f3468430-3cb1-4d6e-8fbc-21329270417f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>